In [ ]:
import numpy as np
import random
import time
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class Team:
  def __init__(self,self_features,opponent_features):
    self.self_features = self_features
    self.opponent_features = opponent_features
    self.explore_wicket = 0
    self.explore_runs = 0

  def get_next_batter(self, order,wicket):
    order = {k:v.tolist() for k,v in order.items()}
    order = dict(sorted(order.items(), key=lambda item: item[wicket]))
    next_batter = list(order)[-1]
    del order[next_batter]
    return next_batter

  def get_next_bowler(self):
    return np.random.randint(0,5)

  def get_batting_action(self,ball,total_runs,wickets_left,score_to_chase,feature_batter,feature_bowler): # code for UCTS
    batting_action = np.random.randint(0,6)
    return batting_action

  def get_bowling_action(self,ball,total_runs,wickets_left,score_to_chase,feature_batter,feature_bowler): # code for UCTS
    bowling_action = np.random.randint(0,3)
    return bowling_action

  def get_explore_actions(self):
    batter = np.random.randint(0,5)
    bowler = np.random.randint(0,5)
    batting_action = np.random.randint(0,6)
    bowling_action = np.random.randint(0,3)
    feature_batter = self.self_features[batter,0:2]
    feature_bowler = self.self_features[bowler,2:4]
    return feature_batter,feature_bowler, batting_action, bowling_action ,batter,bowler 

  def set_explore_outcomes(self,wicket,runs):
    self.explore_wicket = wicket
    self.explore_runs = runs
  
  def explore_compute(self):
    pass

  def explore_dp(self):
    pass


   

		
class Australia(Team):
  pass

class India(Team):
  pass


In [ ]:
class Environment: 
    def get_outcome(self,feature_batter, feature_bowler, batting_action, bowling_action):
      wicket = np.random.randint(0,2)
      runs = np.random.randint(0,7)
      return wicket, runs

class Match:

  def __init__(self,num_balls, explore_num_balls,action_timeout, explore_timeout, TeamOne, TeamTwo):
    self.environment = Environment()
    self.num_balls = num_balls
    self.explore_num_balls = explore_num_balls
    self.action_timeout = action_timeout
    self.explore_timeout = explore_timeout
    self.team_one_features = np.random.uniform(1,5,size=(5,4))
    self.team_two_features = np.random.uniform(1,5,size=(5,4))
    self.team_one = TeamOne(self.team_one_features,self.team_two_features)
    self.team_two = TeamTwo(self.team_two_features,self.team_one_features)
    self.current_batters_list =np.array([1,1,1,1,1])
    self.current_bowlers_list =np.array([2,2,2,2,2])
    self.num_miss_team_batting = 0
    self.num_miss_team_bowling = 0
    self.order = dict.fromkeys([0,1,2,3,4],0)



  def explore_phase_team(self,team_id):
    if (team_id == 1):
      team = self.team_one
    else:
      team = self.team_two
    start_time = time.time()
    
    for balls in range(self.explore_num_balls):
      feature_batter,feature_bowler, batting_action, bowling_action,batter,bowler = team.get_explore_actions()
      wicket, runs = self.get_outcome_random(feature_batter, feature_bowler, batting_action, bowling_action)
      #wicket,runs,p_out,p_runs = self.get_outcome_lin(feature_batter, feature_bowler, batting_action, bowling_action)

      max_balls = 60
      max_wickets = 5
      max_v = 0
      V=np.zeros((max_balls,max_wickets),dtype=float)
      Q=np.zeros((max_balls,max_wickets,5),dtype=float)
      V[max_balls-1,:] = self.Calculate_Value(np.zeros((max_wickets),dtype=float), feature_batter, feature_bowler)
      for i in range(max_balls-2,-1,-1):
        V[i,:]=self.Calculate_Value(V[i+1,:], feature_batter, feature_bowler)
      self.order[batter] += sum(V)/300

      
      team.set_explore_outcomes (wicket,runs)
      team.explore_compute()
    
    team.explore_dp()    
    end_time = time.time()    
    if(end_time - start_time > self.explore_timeout):  
      print("Timing Violation During Exploration Phase")
    return self.order


  def explore_phase(self):
    order1 = self.explore_phase_team(1)
    order2 = self.explore_phase_team(2)
    return order1,order2


  def get_valid_bowler(self,next_bowler):
    if (self.current_bowlers_list[next_bowler]==0):
      print("Bowler Invalid, Choosing Random Bowler")
      bowlers_with_overs_left = np.where(self.current_bowlers_list>0)[0]
      next_bowler = np.random.choice(bowlers_with_overs_left)
    return next_bowler


  def get_valid_batter(self,next_batter):
    if (self.current_batters_list[next_batter]==0):
      print("Batter Invalid, Choosing Random Batter")
      batters_not_out = np.where(self.current_batters_list>0)[0]
      next_batter = np.random.choice(batters_not_out)
    return next_batter

  def next_batter(self,team_batting, order,wicket):
    next_batter = team_batting.get_next_batter(order,wicket)
    next_batter = self.get_valid_batter(next_batter)
    feature_batter = team_batting.self_features[next_batter,0:2]
    return next_batter,feature_batter


  def next_bowler(self,team_bowling):
    next_bowler = team_bowling.get_next_bowler()
    next_bowler = self.get_valid_bowler(next_bowler)
    feature_bowler = team_bowling.self_features[next_bowler,2:4]
    return next_bowler, feature_bowler

  def get_team_batting_action(self,team_batting,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler):
    start_time      = time. time()
    batting_action  = team_batting.get_batting_action(ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler)
    end_time        = time. time()
    if(end_time - start_time > self.action_timeout):
      batting_action = 0 #this is the default option, we have to fix the penalisation strategy
      self.num_miss_team_batting = self.num_miss_team_batting + 1
    return batting_action


  def get_team_bowling_action(self,team_bowling,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler):
    start_time      = time. time()
    bowling_action  = team_bowling.get_bowling_action(ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler)
    end_time        = time. time()
    if(end_time - start_time > self.action_timeout):
      bowling_action = 0 #this is the default option, we have to fix the penalisation strategy
      self.num_miss_team_bowling = self.num_miss_team_bowling + 1
    return bowling_action

  def get_outcome_lin(self,feature_batter, feature_bowler, batting_action, bowling_action):
    wicket = np.random.randint(0,1)
    runs = np.random.randint(0,7)
    p_out = ((batting_action+0.05)*(bowling_action+0.05)*feature_batter[0])/(feature_bowler[0]+62)
    p_runs = ((feature_bowler[1]+0.05)/((bowling_action+5)*feature_batter[1])) - 0.01
    est_wicket = p_out*np.random.rand()*2
    if est_wicket>0.6:
      wicket = 1
    else:
      wicket = 0
    est_runs = p_runs*batting_action*np.random.rand()*2
    runs = math.floor(est_runs)
    return wicket, runs,p_out, p_runs
  def get_outcome_random(self, feature_batter, feature_bowler, batting_action, bowling_action): # Code the outcome function here
    
    # print(self.p_out(feature_batter[0], feature_bowler[0]))
    p_w = ((batting_action+0.05)*(bowling_action+0.05)*feature_batter[0])/(feature_bowler[0]+62)#self.p_out(feature_batter[0], feature_bowler[0])[bowling_action, batting_action] # batting_avg, bowling_avg ###
    
    p_r = ((feature_bowler[1]+0.05)/((bowling_action+5)*feature_batter[1])) - 0.01#self.p_run(feature_batter[1], feature_bowler[1])[bowling_action] # strike rate and economy
    #t_wicket = p_w*np.random.rand()*2
    #if t_wicket>0.6:
      #wicket = 1
    #else:
      #wicket = 0
    #est_runs = p_r*batting_action*np.random.rand()*2
    #runs = math.floor(est_runs)
    runs = 0
    wicket = random.choices([1,0], cum_weights = (p_w, 1))[0]
    if wicket == 0:
      runs = batting_action*(random.choices([1,0], cum_weights = (p_r, 1))[0])
    

    return wicket, runs





     
  def Calculate_Value(self, V_in, feature_batter, feature_bowler):
    #mcts
    runs=[0,1,2,3,4,6]
    max_wickets = 5
    Q_out=np.zeros((max_wickets,6),dtype=float)
    V_out=np.zeros(np.size(V_in),dtype=float) # max wickets size
    shot=np.zeros(np.size(V_in),dtype=int)

    for i in range(1,np.size(V_in)):	
      for a in range(np.size(runs)):
        x = np.random.randint(0,3)
        p_w = ((x+0.05)*(a+0.05)*feature_batter[0])/(feature_bowler[0]+62)#self.p_out(feature_batter[0], feature_bowler[0])[x, a] # batting features, bowling features, batting_action, bowling_action ###
        p_r = ((feature_bowler[1]+0.05)/((a+5)*feature_batter[1])) - 0.01#self.p_run(feature_batter[1], feature_bowler[1])[x] 
        # a being runs
        Q_out[i][a]=(1-p_w)*p_r*runs[a]+p_w*V_in[i-1]+(1-p_w)*V_in[i]
      V_out[i]=np.max(Q_out[i,:])
      shot[i]=runs[np.argmax(Q_out[i,:])]
    return V_out 


  def innings(self,innigins_id,runs_to_chase, order):
    total_runs = 0
    wickets_left = 5
    self.current_batters_list =np.array([1,1,1,1,1])
    self.current_bowlers_list =np.array([2,2,2,2,2])
    self.num_miss_team_batting = 0
    self.num_miss_team_bowling = 0
    if (innigins_id == 1 ):
      team_batting = self.team_one
      team_bowling = self.team_two
    else:
      team_batting = self.team_twoorder
      team_bowling = self.team_one
    # Initialising the first batter and first bowler
    batter,feature_batter = self.next_batter(team_batting,order,5-wickets_left)
    bowler, feature_bowler = self.next_bowler(team_bowling)
    
    for ball in range(self.num_balls):
      if np.sum(self.current_batters_list) > 0 :
        batting_action = self.get_team_batting_action(team_batting,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler)
        bowling_action = self.get_team_bowling_action(team_bowling,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler)
        wicket, runs   = self.get_outcome_random(feature_batter, feature_bowler, batting_action, bowling_action)
        #wicket, runs   = self.environment.get_outcome(feature_batter, feature_bowler, batting_action, bowling_action)
        total_runs     = total_runs + runs
        if (wicket > 0):
          self.current_batters_list[batter] = 0
          if(np.sum(self.current_batters_list) > 0 ):
            batter,feature_batter = self.next_batter(team_batting,order,wicket)
        if ((ball+1)%6 ==0 ):
          self.current_bowlers_list[bowler] = self.current_bowlers_list[bowler]-1
          if(np.sum(self.current_bowlers_list) > 0 ) :
            bowler, feature_bowler = self.next_bowler(team_bowling)

    return total_runs, self.current_batters_list, self.current_bowlers_list, self.num_miss_team_batting, self.num_miss_team_bowling


In [ ]:
# setting time outs
explore_timeout = 100
action_timeout = 100

explore_num_balls = 60
num_balls = 60
match = Match(num_balls, explore_num_balls,action_timeout, explore_timeout,Australia,India)
order1,order2= match.explore_phase()

# mcts(runs = 0, wickets = 0, order)
first_innings_score, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(1, float('inf'),order1)
print(first_innings_score)
#second_innings_score, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(2,first_innings_score)

Batter Invalid, Choosing Random Batter
Batter Invalid, Choosing Random Batter
Batter Invalid, Choosing Random Batter
Bowler Invalid, Choosing Random Bowler
Bowler Invalid, Choosing Random Bowler
35
